In [2]:
import pandas as pd
import sqlite3

In [3]:
con=sqlite3.connect("terrorism.db")

In [4]:
cleaned_data=pd.read_csv("cleaned_global_terrorism.csv")
cleaned_data.to_sql("terrorism_data", con, if_exists='replace', index=False)

181691

In [5]:
new_data=pd.read_sql("""select * from terrorism_data""",con)
new_data

,Unnamed: 0,eventid,iyear,imonth,iday,country_txt,region_txt,attacktype1_txt,nkill,nwound,targtype1_txt,weaptype1_txt,casualities
0,0,1.970000e+11,1970,7,2,Dominican Republic,Central America & Caribbean,Assassination,1.0,0.0,Private Citizens & Property,Unknown,1.0
1,1,1.970000e+11,1970,0,0,Mexico,North America,Hostage Taking (Kidnapping),0.0,0.0,Government (Diplomatic),Unknown,0.0
2,2,1.970010e+11,1970,1,0,Philippines,Southeast Asia,Assassination,1.0,0.0,Journalists & Media,Unknown,1.0
3,3,1.970010e+11,1970,1,0,Greece,Western Europe,Bombing/Explosion,0.0,0.0,Government (Diplomatic),Explosives,0.0
4,4,1.970010e+11,1970,1,0,Japan,East Asia,Facility/Infrastructure Attack,0.0,0.0,Government (Diplomatic),Incendiary,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,181686,2.017120e+11,2017,12,31,Somalia,Sub-Saharan Africa,Armed Assault,1.0,2.0,Military,Firearms,3.0
181687,181687,2.017120e+11,2017,12,31,Syria,Middle East & North Africa,Bombing/Explosion,2.0,7.0,Military,Explosives,9.0
181688,181688,2.017120e+11,2017,12,31,Philippines,Southeast Asia,Facility/Infrastructure Attack,0.0,0.0,Private Citizens & Property,Incendiary,0.0
181689,181689,2.017120e+11,2017,12,31,India,South Asia,Bombing/Explosion,0.0,0.0,Government (General),Explosives,0.0


In [6]:
#Most Affected Countries
new_data=pd.read_sql("""SELECT country_txt, SUM(casualities) AS total_casualties 
FROM terrorism_data 
GROUP BY country_txt 
ORDER BY total_casualties DESC 
LIMIT 10""",con)
new_data

,country_txt,total_casualties
0,Iraq,213279.0
1,Afghanistan,83661.0
2,Pakistan,65860.0
3,India,48321.0
4,Nigeria,32921.0
5,Sri Lanka,31091.0
6,Syria,29338.0
7,Colombia,25026.0
8,United States,24473.0
9,Philippines,22926.0


In [7]:
#average casualities for each attack type
new_data=pd.read_sql("""SELECT attacktype1_txt, round(avg(casualities),2) AS avg_casualties 
FROM terrorism_data 
GROUP BY 1 
ORDER BY 2 DESC""",con)
new_data

,attacktype1_txt,avg_casualties
0,Hijacking,31.44
1,Unarmed Assault,14.69
2,Hostage Taking (Barricade Incident),8.52
3,Unknown,6.47
4,Bombing/Explosion,6.01
5,Armed Assault,5.57
6,Hostage Taking (Kidnapping),2.75
7,Assassination,2.01
8,Facility/Infrastructure Attack,0.72


In [8]:
#total casualities per region
new_data=pd.read_sql("""SELECT region_txt, SUM(casualities) AS total_casualties, sum(nkill) as total_killed
FROM terrorism_data 
GROUP BY 1 
ORDER BY total_casualties DESC""",con)
new_data

,region_txt,total_casualties,total_killed
0,Middle East & North Africa,351950.0,137642.0
1,South Asia,242679.0,101319.0
2,Sub-Saharan Africa,131243.0,78386.0
3,South America,45553.0,28849.0
4,Southeast Asia,41896.0,15637.0
5,Central America & Caribbean,37699.0,28708.0
6,North America,26447.0,4916.0
7,Western Europe,25026.0,6694.0
8,Eastern Europe,19460.0,7415.0
9,East Asia,10365.0,1152.0


In [9]:
#total killed per region based on year and month
new_data=pd.read_sql("""SELECT region_txt, SUM(casualities) AS total_casualties, sum(nkill) as total_killed
FROM terrorism_data
WHERE iyear='2000' and imonth='7'                
GROUP BY 1 
ORDER BY 2 DESC""",con)
new_data

,region_txt,total_casualties,total_killed
0,Eastern Europe,227.0,89.0
1,South Asia,192.0,80.0
2,Southeast Asia,177.0,64.0
3,Middle East & North Africa,98.0,86.0
4,Sub-Saharan Africa,87.0,44.0
5,South America,61.0,28.0
6,Western Europe,37.0,2.0
7,Central Asia,5.0,1.0
8,North America,4.0,0.0
9,Central America & Caribbean,2.0,2.0


In [10]:
#various targets and weapons used to kill
new_data=pd.read_sql("""SELECT targtype1_txt, weaptype1_txt,sum(nkill) as total_killed
FROM terrorism_data              
GROUP BY 1,2
ORDER BY 3 DESC""",con)
new_data

,targtype1_txt,weaptype1_txt,total_killed
0,Private Citizens & Property,Firearms,58421.0
1,Military,Firearms,52731.0
2,Private Citizens & Property,Explosives,51918.0
3,Military,Explosives,39644.0
4,Police,Firearms,25228.0
...,...,...,...
183,Transportation,Radiological,0.0
184,Unknown,Sabotage Equipment,0.0
185,Utilities,Melee,0.0
186,Utilities,Other,0.0
